<a href="https://colab.research.google.com/github/ereinha/SineKAN/blob/main/MLP_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
class MLP(nn.Module):
    def __init__(self, layers_hidden):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers_hidden) - 1):
            self.layers.append(nn.Linear(layers_hidden[i], layers_hidden[i + 1]))
            if i < len(layers_hidden) - 2:
                self.layers.append(nn.ReLU())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [4]:
# Load MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
valset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
train_loader = DataLoader(trainset, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(valset, batch_size=64, num_workers=2, shuffle=False)

In [ ]:
epochs = 30
lrs = [8e-4, 1.2e-3, 1.6e-3]
gammas = [0.8, 0.9]
hdims = [64, 128, 256]
best_accs = []
for lr in lrs:
    for gamma in gammas:
        for hdim in hdims:
            torch.manual_seed(42)
            best_acc = 0
            # Define model
            model = MLP(layers_hidden=[28 * 28, hdim, 10])
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            # Define optimizer
            optimizer = optim.AdamW(model.parameters(), lr=lr)
            # Define learning rate scheduler
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

            # Define loss
            criterion = nn.CrossEntropyLoss()
            for epoch in range(epochs):
                # Train
                model.train()
                with tqdm(train_loader) as pbar:
                    for i, (images, labels) in enumerate(pbar):
                        images = images.view(-1, 28 * 28).to(device)
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels.to(device))
                        loss.backward()
                        optimizer.step()
                        accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                        pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

                # Validation
                model.eval()
                val_loss = 0
                val_accuracy = 0
                with torch.no_grad():
                    for images, labels in val_loader:
                        images = images.view(-1, 28 * 28).to(device)
                        output = model(images)
                        val_loss += criterion(output, labels.to(device)).item()
                        val_accuracy += (
                            (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                        )
                val_loss /= len(val_loader)
                val_accuracy /= len(val_loader)
                if val_accuracy > best_acc:
                    best_acc = val_accuracy

                # Update learning rate
                scheduler.step()

                print(
                    f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
                )
            best_accs.append(best_acc)
            print(f"LR: {lr} Gamma: {gamma} Hdim: {hdim} Best Accuracy: {best_acc}")

100%|██████████| 938/938 [00:40<00:00, 23.17it/s, accuracy=0.938, loss=0.246, lr=0.0008]


Epoch 1, Val Loss: 0.2780421424099499, Val Accuracy: 0.9196855095541401


100%|██████████| 938/938 [00:39<00:00, 23.65it/s, accuracy=0.906, loss=0.282, lr=0.00064]


Epoch 2, Val Loss: 0.24632722592562628, Val Accuracy: 0.9284434713375797


100%|██████████| 938/938 [00:39<00:00, 23.71it/s, accuracy=0.938, loss=0.14, lr=0.000512]


...

Epoch 28, Val Loss: 0.07196281746632993, Val Accuracy: 0.9815883757961783


100%|██████████| 938/938 [00:25<00:00, 37.44it/s, accuracy=1, loss=0.00214, lr=8.37e-5]


Epoch 29, Val Loss: 0.07209294401118285, Val Accuracy: 0.9812898089171974


100%|██████████| 938/938 [00:24<00:00, 38.71it/s, accuracy=1, loss=0.000476, lr=7.54e-5]


Epoch 30, Val Loss: 0.07353632927119624, Val Accuracy: 0.9811902866242038
LR: 0.0016 Gamma: 0.9 Hdim: 256 Best Accuracy: 0.981687898089172


In [ ]:
epochs = 30
wds = [0.01, 0.1, 0.25, 0.5, 0.75, 1.]
wd_accs = np.empty((6, epochs))
for h, wd in enumerate(wds):
    torch.manual_seed(42)
    best_acc = 0
    # Define model
    model = MLP(layers_hidden=[28 * 28, 64, 10])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=8e-4, weight_decay=wd)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                val_accuracy += (
                    (output.argmax(dim=1) == labels.to(device)).float().mean().item()
                )

        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        wd_accs[h, epoch] = val_accuracy

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:19<00:00, 47.86it/s, accuracy=0.938, loss=0.22, lr=0.0008]


Epoch 1, Val Loss: 0.1956070644401346, Val Accuracy: 0.9424761146496815


100%|██████████| 938/938 [00:19<00:00, 48.55it/s, accuracy=0.938, loss=0.136, lr=0.00072]


Epoch 2, Val Loss: 0.1331768139639193, Val Accuracy: 0.9617834394904459


100%|██████████| 938/938 [00:18<00:00, 49.86it/s, accuracy=0.938, loss=0.279, lr=0.000648]


...

Epoch 28, Val Loss: 0.1076340474704411, Val Accuracy: 0.973328025477707


100%|██████████| 938/938 [00:19<00:00, 47.73it/s, accuracy=1, loss=0.0822, lr=4.19e-5]


Epoch 29, Val Loss: 0.10777284540117356, Val Accuracy: 0.9734275477707006


100%|██████████| 938/938 [00:19<00:00, 49.33it/s, accuracy=1, loss=0.0322, lr=3.77e-5]


Epoch 30, Val Loss: 0.10801566690562447, Val Accuracy: 0.9728304140127388


In [ ]:
epochs = 30
hdims = [16, 32, 64, 128, 256, 512, 1024, 2048]
hdim_metrics = {
    'accuracy': np.empty((8, epochs)),
    'precision': np.empty((8, epochs)),
    'recall': np.empty((8, epochs)),
    'f1_score': np.empty((8, epochs)),
}
for h, hdim in enumerate(hdims):
    torch.manual_seed(42)
    # Define model
    model = MLP(layers_hidden=[28 * 28, hdim, 10])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=8e-4, weight_decay=0.01)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        hdim_metrics['accuracy'][h, epoch] = val_accuracy
        hdim_metrics['precision'][h, epoch] = val_precision
        hdim_metrics['recall'][h, epoch] = val_recall
        hdim_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, "
            f"Val Accuracy: {val_accuracy}, Precision: {val_precision}, "
            f"Recall: {val_recall}, F1 Score: {val_f1}"
        )

100%|██████████| 938/938 [00:25<00:00, 36.71it/s, accuracy=0.812, loss=0.406, lr=0.0008]


Epoch 1, Val Loss: 0.4509099055627349, Val Accuracy: 0.8728, Precision: 0.8754268308847603, Recall: 0.8728, F1 Score: 0.8727474901024427


100%|██████████| 938/938 [00:22<00:00, 41.17it/s, accuracy=0.938, loss=0.206, lr=0.00072]


Epoch 2, Val Loss: 0.3821559297336135, Val Accuracy: 0.8939, Precision: 0.8954028198579713, Recall: 0.8939, F1 Score: 0.8942089680062498


100%|██████████| 938/938 [00:18<00:00, 49.51it/s, accuracy=0.906, loss=0.23, lr=0.000648]


...

Epoch 28, Val Loss: 0.0627610383297782, Val Accuracy: 0.9863, Precision: 0.9863009246068559, Recall: 0.9863, F1 Score: 0.9862977104352781


100%|██████████| 938/938 [00:18<00:00, 51.47it/s, accuracy=1, loss=0.000205, lr=4.19e-5]


Epoch 29, Val Loss: 0.062441392226461286, Val Accuracy: 0.9863, Precision: 0.9863051193893114, Recall: 0.9863, F1 Score: 0.9862984867921796


100%|██████████| 938/938 [00:18<00:00, 51.24it/s, accuracy=1, loss=0.00024, lr=3.77e-5]


Epoch 30, Val Loss: 0.062361313227263654, Val Accuracy: 0.9856, Precision: 0.9856030342467363, Recall: 0.9856, F1 Score: 0.9855978293941098


In [ ]:
np.save('./saved_results/mlp_hdim_metrics.npy', hdim_metrics)

In [ ]:
epochs = 30
n_layers = [1, 2, 3, 4]
nlayer_metrics = {
    'accuracy': np.empty((4, epochs)),
    'precision': np.empty((4, epochs)),
    'recall': np.empty((4, epochs)),
    'f1_score': np.empty((4, epochs)),
}
for h, n_layer in enumerate(n_layers):
    torch.manual_seed(42)
    # Define model
    model = MLP(layers_hidden=[28 * 28] + [128]*n_layer + [10])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=8e-4, weight_decay=.01)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Define loss
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        # Train
        model.train()
        with tqdm(train_loader) as pbar:
            for i, (images, labels) in enumerate(pbar):
                images = images.view(-1, 28 * 28).to(device)
                optimizer.zero_grad()
                output = model(images)
                loss = criterion(output, labels.to(device))
                loss.backward()
                optimizer.step()
                accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()
                pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, 28 * 28).to(device)
                output = model(images)
                val_loss += criterion(output, labels.to(device)).item()
                preds = output.argmax(dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
        val_precision = precision_score(all_labels, all_preds, average='weighted')
        val_recall = recall_score(all_labels, all_preds, average='weighted')
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        nlayer_metrics['accuracy'][h, epoch] = val_accuracy
        nlayer_metrics['precision'][h, epoch] = val_precision
        nlayer_metrics['recall'][h, epoch] = val_recall
        nlayer_metrics['f1_score'][h, epoch] = val_f1

        # Update learning rate
        scheduler.step()

        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
        )

100%|██████████| 938/938 [00:17<00:00, 53.59it/s, accuracy=0.938, loss=0.134, lr=0.0008]


Epoch 1, Val Loss: 0.24920118076928482, Val Accuracy: 0.9551


100%|██████████| 938/938 [00:17<00:00, 53.40it/s, accuracy=0.969, loss=0.145, lr=0.00072]


Epoch 2, Val Loss: 0.16810530201964982, Val Accuracy: 0.9539


100%|██████████| 938/938 [00:17<00:00, 54.60it/s, accuracy=0.969, loss=0.066, lr=0.000648]


...

Epoch 28, Val Loss: 0.09855211519892938, Val Accuracy: 0.9740865546218488


100%|██████████| 938/938 [00:18<00:00, 49.89it/s, accuracy=1, loss=0.000298, lr=4.19e-5]


Epoch 29, Val Loss: 0.10230289873200406, Val Accuracy: 0.9741375


100%|██████████| 938/938 [00:20<00:00, 46.18it/s, accuracy=1, loss=0.000177, lr=3.77e-5]


Epoch 30, Val Loss: 0.101977787743816, Val Accuracy: 0.9741917355371901


In [ ]:
np.save('./saved_results/mlp_layer_metrics.npy', nlayer_metrics)

In [5]:
from torch.profiler import profile, record_function, ProfilerActivity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def profile_model(model, inputs, num_runs=100):
    total_times = []
    for _ in range(num_runs):
        total_time = 0
        with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
            model(inputs)

        # Extract total CUDA time spent on 'model_inference'
        for event in prof.key_averages():
            total_time += event.device_time

        total_times.append(total_time)
    average_time = np.mean(total_times)
    std_time = np.std(total_times)
    print(f"Average inference time: {average_time} microseconds +/- {std_time}")
    return average_time, std_time

In [7]:
inputs = torch.randn((1, 784)).to(device)
hdims = [16, 32, 64, 128, 256, 512, 1024, 2048]
means = []
stds = []
for h in hdims:
    model = MLP(layers_hidden=[28 * 28, h, 10]).to(device)
    print('Hidden Dims: %d' % h)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Hidden Dims: 16
Average inference time: 16.469300000000068 microseconds +/- 0.1294072100017815
Hidden Dims: 32
Average inference time: 18.38102800000001 microseconds +/- 0.0591990812091114
Hidden Dims: 64
Average inference time: 15.246054999999982 microseconds +/- 0.1816489635946277
Hidden Dims: 128
Average inference time: 15.652834999999985 microseconds +/- 0.14793943955214972
Hidden Dims: 256
Average inference time: 17.532512000000004 microseconds +/- 0.06608825807962573
Hidden Dims: 512
Average inference time: 22.471773 microseconds +/- 0.13574125928029704
Hidden Dims: 1024
Average inference time: 30.496041000000005 microseconds +/- 0.12915276736872705
Hidden Dims: 2048
Average inference time: 46.71211000000001 microseconds +/- 0.33805827293530394


In [8]:
np.save('./saved_results/mlp_scaling_mean_hdims.npy', means)
np.save('./saved_results/mlp_scaling_std_hdims.npy', stds)

In [9]:
hdims = [16, 32, 64, 128, 256, 512, 1024, 2048]
means = []
stds = []
for h in hdims:
    model = MLP(layers_hidden=[28 * 28, 64, 10]).to(device)
    inputs = torch.randn((h, 784)).to(device)
    print('Input Batch Size: %d' % h)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Input Batch Size: 16
Average inference time: 14.677882000000002 microseconds +/- 0.03578950371267998
Input Batch Size: 32
Average inference time: 24.0251725 microseconds +/- 0.04129802045801046
Input Batch Size: 64
Average inference time: 23.151859999999992 microseconds +/- 0.041839280586549024
Input Batch Size: 128
Average inference time: 43.249322 microseconds +/- 0.12244278792970538
Input Batch Size: 256
Average inference time: 48.577977 microseconds +/- 0.20088813919940596
Input Batch Size: 512
Average inference time: 64.616773 microseconds +/- 0.1973470634972756
Input Batch Size: 1024
Average inference time: 107.24766500000001 microseconds +/- 0.36356909766232276
Input Batch Size: 2048
Average inference time: 165.409793 microseconds +/- 1.003110950070331


In [10]:
np.save('./saved_results/mlp_scaling_mean_batch.npy', means)
np.save('./saved_results/mlp_scaling_std_batch.npy', stds)

In [11]:
nlayers = [1, 2, 3, 4]
means = []
stds = []
for n in nlayers:
    model = MLP(layers_hidden=[28 * 28] + [64]*n +[10]).to(device)
    inputs = torch.randn((1, 784)).to(device)
    print('Number of Hidden Layers: %d' % n)
    avg, std = profile_model(model, inputs, 1000)
    means.append(avg)
    stds.append(std)

Number of Hidden Layers: 1
Average inference time: 14.979579000000003 microseconds +/- 0.18191051580103915
Number of Hidden Layers: 2
Average inference time: 19.859807999999997 microseconds +/- 0.21762052553929087
Number of Hidden Layers: 3
Average inference time: 19.734595666666667 microseconds +/- 0.1643495418412975
Number of Hidden Layers: 4
Average inference time: 19.711927000000003 microseconds +/- 0.22946895128002276


In [13]:
np.save('./saved_results/mlp_scaling_mean_layers.npy', means)
np.save('./saved_results/mlp_scaling_std_layers.npy', stds)